# BERT

In [1]:
import torch
import math
from d2l import torch as d2l
from torch import nn

In [2]:
def transpose_qkv(X, num_heads):
    """为了多注意力头的并行计算而变换形状"""
    # 输入X的形状:(batch_size，查询或者“键－值”对的个数，num_hiddens)
    # 输出X的形状:(batch_size，查询或者“键－值”对的个数，num_heads， num_hiddens/num_heads)
    X= X.reshape(X.shape[0], X.shape[1], num_heads, -1)
    # 输出X的形状:(batch_size，num_heads，查询或者“键－值”对的个数, num_hiddens/num_heads)
    X = X.permute(0, 2, 1, 3)
    # 最终输出的形状:(batch_size*num_heads,查询或者“键－值”对的个数, num_hiddens/num_heads)
    return X.reshape(-1, X.shape[2], X.shape[3])

#@save
def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

class MultiHeaderAttention(nn.Module):
    def __init__(self, key_size, query_size, value_size, num_hiddens, num_heads, dropout, bias=False) -> None:
        super().__init__()
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)
    
    def forward(self, queries, keys, values, valid_lens):
        # queries，keys，values的形状:
        # (batch_size，查询或者“键－值”对的个数，num_hiddens)
        # valid_lens　的形状:
        # (batch_size，)或(batch_size，查询的个数)
        # 经过变换后，输出的queries，keys，values　的形状:
        # (batch_size*num_heads，查询或者“键－值”对的个数，num_hiddens/num_heads)
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)
        
        if valid_lens is not None:
            # 将 valid_lens 重复 num_heads 次，因为每个注意力头都需要独立的 valid_lens
            # 注意：参数名是 repeats 而不是 repeat
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0
            )
        
        output = self.attention(queries, keys, values, valid_lens)
        
        output_cat = transpose_output(output, num_heads=self.num_heads)
        return self.W_o(output_cat)

class AddNorm(nn.Module):
    def __init__(self, normalized_shape, dropout) -> None:
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)
    
    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)

class PositionWiseFFN(nn.Module):
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs) -> None:
        super().__init__()
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)
    
    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))

#@save
class EncoderBlock(nn.Module):
    """Transformer编码器块"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                 dropout, use_bias=False, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        # 使用我们自己定义的 MultiHeaderAttention，而不是 d2l.MultiHeadAttention
        # 因为 d2l.MultiHeadAttention 的签名可能不同
        self.attention = MultiHeaderAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout,
            bias=use_bias)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFFN(
            ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)

    def forward(self, X, valid_lens):
        Y = self.addnorm1(X, self.attention(X, X, X, valid_lens))
        return self.addnorm2(Y, self.ffn(Y))

In [3]:
def get_tokens_and_segments(tokens_a, tokens_b):
    tokens = ['<cls>'] + tokens_a + ['<seq>']
    segments = [0] * (len(tokens_a) + 2)
    if tokens_b is not None:
        tokens += tokens_b + ['<seq>']
        segments += [1] * (len(tokens_b) + 1)
    return tokens, segments

In [4]:
class BertEncoder(nn.Module):
    """
    BertEncoder
    segment表示“句子片段类型embedding”。
    在BERT中，输入通常是两段文本拼接，例如句子A和句子B。
    segment用于区分不同的句子（例如A为0，B为1），以便模型能够知道某个token属于哪一部分。

    输入是2，代表segment可以取两种类型（0或1）：0表示第一个句子片段，1表示第二个句子片段。
    如果只输入单句任务，全部segment为0；如果是句子对任务，根据分割点设置为0和1。
    """
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input,
                 ffn_num_hiddens, num_heads, num_layers, dropout,
                 max_lens = 1000, key_size=768, query_size=768, value_size=768) -> None:
        super().__init__()
        self.token_embeding = nn.Embedding(vocab_size, num_hiddens)
        # segment_embeding输入2，代表两种类型（句子1和句子2：0或1）
        self.segment_embeding = nn.Embedding(2, num_hiddens)
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module(f"{i}", EncoderBlock(key_size, query_size, value_size, num_hiddens, norm_shape,
                                                      ffn_num_input, ffn_num_hiddens, num_heads, dropout, True))
        # 位置编码，shape=[1, max_lens, num_hiddens]，可学习
        self.pos_embeding = nn.Parameter(torch.randn(1, max_lens, num_hiddens))
    
    def forward(self, token, segment, valid_lens):
        """
        token: 词索引序列，[batch, seq_len]
        segment: 句子片段类型，[batch, seq_len]，值为0或1
        valid_lens: 有效长度
        """
        # token embedding + segment embedding
        X = self.token_embeding(token) + self.segment_embeding(segment)
        # 加上可学习的位置编码
        X = X + self.pos_embeding.data[:, :X.shape[1], :]
        for blk in self.blks:
            X = blk(X, valid_lens)
        return X

In [5]:
# 会报错的原因：
# nn.Embedding(2,3) 代表只能索引0或1（即num_embeddings=2，对应词表id只能是0/1），但x里有2，超出词表范围
a = nn.Embedding(11, 3)
x = torch.tensor([1, 10, 1, 2, 1])
try:
    out = a(x)
    print(out)
except Exception as e:
    print("出错了:", e)

tensor([[-1.5456, -1.1562,  1.4382],
        [-1.4969,  1.3885, -0.5923],
        [-1.5456, -1.1562,  1.4382],
        [-0.5804, -0.6946, -1.1193],
        [-1.5456, -1.1562,  1.4382]], grad_fn=<EmbeddingBackward0>)


In [6]:
vocab_size, num_hiddens, ffn_num_hiddens, num_heads = 10000, 768, 1024, 4
norm_shape, ffn_num_input, num_layers, dropout = [768], 768, 2, 0.2
encoder = BertEncoder(vocab_size, num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, num_layers, dropout)

In [7]:
tokens = torch.randint(0, vocab_size, (2,8))
print(tokens)

tensor([[4586, 7666, 7013, 8943, 8274, 4586, 4618, 2260],
        [5724,  561, 3460, 5897, 4056, 7071, 4400, 7349]])


In [8]:
segments = torch.tensor([[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1]])
encoder_X = encoder(tokens, segments, None)
encoder_X.shape

torch.Size([2, 8, 768])

In [9]:
class MaskLM(nn.Module):
    def __init__(self, vocab_size, num_hiddens, num_inputs=768) -> None:
        """
        Masked Language Model（MLM）模块。

        参数说明：
        vocab_size: 词表大小，输出类别数（即预测每个位置对应的词汇表token）。
        num_hiddens: 隐藏层的维度。
        num_inputs: 输入特征的维度，通常等于BERT编码器输出的隐藏单元数，默认768。
        """
        super().__init__()
        # 构造一个MLP（多层感知机），输入num_inputs维，经过隐藏层后输出vocab_size维的预测。
        self.mlp = nn.Sequential(
            nn.Linear(num_inputs, num_hiddens), # 线性变换到隐藏层
            nn.ReLU(),                          # 激活函数
            nn.LayerNorm(num_hiddens),          # 层归一化
            nn.Linear(num_hiddens, vocab_size)  # 映射到vocab_size，为softmax前的logits
        )
    
    def forward(self, X, pred_positions):
        """
        前向传播。

        参数：
        X: 经过BERT encoder后的表示，形状为 [batch_size, seq_len, hidden_dim]，
           表示每个token的上下文表示。
        pred_positions: 要预测的masked位置索引，形状为 [batch_size, num_pred_positions]，
                        每行是一个样本要预测的位置列表。

        返回：
        mlm_Y_hat: 每个被mask位置的预测结果，
                   形状为 [batch_size, num_pred_positions, vocab_size]。
        """
        # 1. 得到每个样本需要预测的token数量
        num_pred_positions = pred_positions.shape[1]
        # 2. 将pred_positions展平为一维，便于统一索引
        pred_positions_flat = pred_positions.reshape(-1)  # 长度为batch_size * num_pred_positions

        batch_size = X.shape[0]
        # 3. 构造一个batch索引。例如batch_size=2, num_pred_positions=3时，得到[0,0,0,1,1,1]
        batch_idx = torch.arange(0, batch_size).repeat_interleave(num_pred_positions)
        # 这样(X[batch_idx, pred_positions_flat])就取出所有需要mask的token的表示

        # 4. 按指定位置收集得到被mask位置的上下文表示，形状为 [batch_size * num_pred_positions, hidden_dim]
        masked_X = X[batch_idx, pred_positions_flat]
        # 5. 恢复成 [batch_size, num_pred_positions, hidden_dim] 的形式
        masked_X = masked_X.reshape((batch_size, num_pred_positions, -1))

        # 6. 通过MLP变换，每个位置最终输出vocab_size维，对应softmax前的logits
        mlm_Y_hat = self.mlp(masked_X)

        # 7. 输出，形状为 [batch_size, num_pred_positions, vocab_size]
        return mlm_Y_hat

In [10]:
mlm = MaskLM(vocab_size, num_hiddens)
mlm_postions = torch.tensor([[1,5,2],[6,1,5]])
mlm_Y_hat = mlm(encoder_X, mlm_postions)
mlm_Y_hat.shape

torch.Size([2, 3, 10000])

In [11]:
mlm_Y = torch.tensor([[7,8,9], [10, 20, 30]])
loss = nn.CrossEntropyLoss(reduction='none')
mlm_l = loss(mlm_Y_hat.reshape((-1, vocab_size)), mlm_Y.reshape(-1))
mlm_l.shape

torch.Size([6])

In [12]:
class NextSentencePred(nn.Module):
    def __init__(self, num_inputs) -> None:
        super().__init__()
        self.output = nn.Linear(num_inputs, 2)
    
    def forward(self, X):
        return self.output(X)

In [13]:
encoder_X = torch.flatten(encoder_X, start_dim=1)
nsp = NextSentencePred(encoder_X.shape[-1])
nsp_Y_hat = nsp(encoder_X)
nsp_Y_hat.shape

torch.Size([2, 2])

In [14]:
nsp_y = torch.tensor([0, 1])
nsp_l = loss(nsp_Y_hat, nsp_y)
nsp_l.shape

torch.Size([2])

In [ ]:
#@save
class BERTModel(nn.Module):
    """BERT模型"""
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input,
                 ffn_num_hiddens, num_heads, num_layers, dropout,
                 max_len=1000, key_size=768, query_size=768, value_size=768,
                 hid_in_features=768, mlm_in_features=768,
                 nsp_in_features=768):
        super(BERTModel, self).__init__()
        self.encoder = BertEncoder(vocab_size, num_hiddens, norm_shape,
                    ffn_num_input, ffn_num_hiddens, num_heads, num_layers,
                    dropout, max_len=max_len, key_size=key_size,
                    query_size=query_size, value_size=value_size)
        self.hidden = nn.Sequential(nn.Linear(hid_in_features, num_hiddens),
                                    nn.Tanh())
        self.mlm = MaskLM(vocab_size, num_hiddens, mlm_in_features)
        self.nsp = NextSentencePred(nsp_in_features)

    def forward(self, tokens, segments, valid_lens=None,
                pred_positions=None):
        encoded_X = self.encoder(tokens, segments, valid_lens)
        if pred_positions is not None:
            mlm_Y_hat = self.mlm(encoded_X, pred_positions)
        else:
            mlm_Y_hat = None
        # 用于下一句预测的多层感知机分类器的隐藏层，0是“<cls>”标记的索引
        nsp_Y_hat = self.nsp(self.hidden(encoded_X[:, 0, :]))
        return encoded_X, mlm_Y_hat, nsp_Y_hat